# Yolov8_Animeface

## A model trained to detect anime faces

We're using comet-ml to monitor the training, and the ultralytics library to download the model. Dataset is made on roboflow and is stored under the "dataset" subfolder of the project.

Dataset not provided in this document.


## Import Monitoring Services

In [ ]:
from comet_ml import Experiment

# Don't turn the code log on if you plan on pasting your api key here.

experiment = Experiment(
    api_key="", # Your API key
    project_name="",
    workspace="" # Your workspace as defined in comet-ml
)


## Train Model

In [ ]:
import torch
# Just making sure cuda is available, if not, check your driver installation.
torch.cuda.is_available()

In [ ]:
from ultralytics import YOLO

# Import the desired model. Look up ultralytics' docs to find out which one you should use. We will use the basic yolov8x here.
yolov8_animeface = YOLO('yolov8x.pt')

results = yolov8_animeface.train(
    data='./dataset/data.yaml',
    epochs=200,
    imgsz=640, # Feel free to tweak imgsz, but be careful as it will be more computally intensive if you put a bigger size.
    batch=16, # Tweak the batch size according to your GPU.
    name='', # Same as your comet project name
    project='', # Same as your comet project name
    single_cls=True, # If you plan on detecting more than faces, remove this line.
    device=[0], # Helpful if your GPU isn't recognized automatically.
    patience=20, # Tweak this as needed
)

Few important points :

We're using the **default SGD optimizer** with automatic values. Adam and RMSProp have both been tried during training but weren't as good as SGD. They were mediocre, even when fidgeting with hyper-parameters.

I have never done more than 100 epoch as my models usually early stop around the ~50th epoch.

**If you have to stop the training for whatever reason and want to resume training :**
1. Replace path in line 4 by the path of the `last.pt` weight previously calculated.
2. Add `resume=True,` inside the training arguments. Be aware that if you keep `resume=True` and try to train a **new** model, it will give you an unrelated CUDA error (Invalid CUDA requested).


In [ ]:
# End comet monitoring
experiment.end()

## Evaluation on test subset

If you've split your dataset properly, you can now do inference on your test sub-dataset.

In [ ]:
eval_model = YOLO('./project/name/weights/best.pt') # Adapt to your own path

metrics = eval_model.val(
    data='./dataset/data.yaml',
    split='test'
)

## Plot results

todo